# CS 267A - Final Project GNN
Use this notebook to test functions collaboratively. 

- https://github.com/fulifeng/Temporal_Relational_Stock_Ranking

## Environment Init 

### Stock and Relational Data

In [ ]:
! git clone https://github.com/fulifeng/Temporal_Relational_Stock_Ranking.git
! tar zxvf Temporal_Relational_Stock_Ranking/data/relation.tar.gz -C Temporal_Relational_Stock_Ranking/data/

Cloning into 'Temporal_Relational_Stock_Ranking'...
remote: Enumerating objects: 9126, done.
remote: Total 9126 (delta 0), reused 0 (delta 0), pack-reused 9126
Receiving objects: 100% (9126/9126), 330.30 MiB | 21.39 MiB/s, done.
Resolving deltas: 100% (46/46), done.
Checking out files: 100% (11376/11376), done.
relation/
relation/sector_industry/
relation/sector_industry/NASDAQ_industry_ticker.json
relation/sector_industry/NYSE_industry_ticker.json
relation/sector_industry/NASDAQ_industry_relation.npy
relation/sector_industry/NYSE_industry_relation.npy
relation/wikidata/
relation/wikidata/NASDAQ_wiki_relation.npy
relation/wikidata/NYSE_wiki_relation.npy
relation/wikidata/NYSE_connections.json
relation/wikidata/NASDAQ_connections.json
relation/wikidata/selected_wiki_connections.csv


### Pretrained Model Weights

In [ ]:
# causes colab to crash. You'll just have to download the pretrained model at
# https://drive.google.com/file/d/1fyNCZ62pEItTQYEBzLwsZ9ehX_-Ai3qT
# I've uploaded it to my Google Drive and mount it to colab to acccess

# from google.colab import auth
# auth.authenticate_user()
# from googleapiclient.discovery import build
# drive_service = build('drive', 'v3')

# file_id = "1fyNCZ62pEItTQYEBzLwsZ9ehX_-Ai3qT"

# import io
# from googleapiclient.http import MediaIoBaseDownload

# request = drive_service.files().get_media(fileId=file_id)
# downloaded = io.BytesIO()
# downloader = MediaIoBaseDownload(downloaded, request)
# done = False
# while done is False:
#   _, done = downloader.next_chunk()

# downloaded.seek(0)
# print('Downloaded file contents are: {}'.format(downloaded.read()))

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
! tar zxvf drive/My\ Drive/data/gnn_stocks/pretrain.tar.gz -C Temporal_Relational_Stock_Ranking/data

tar (child): drive/My Drive/data/gnn_stocks/pretrain.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
% cd Temporal_Relational_Stock_Ranking/training

/content/Temporal_Relational_Stock_Ranking/training/Temporal_Relational_Stock_Ranking/training


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

evaluator.py  load_data.py  __pycache__  rank_lstm.py  relation_rank_lstm.py


## Data Exploration

In [ ]:
import numpy as np
import tensorflow as tf
import os

from load_data import load_EOD_data, load_relation_data

`python relation_rank_lstm.py -rn wikidata -l 16 -u 64 -a 0.1`

### Stock Features

In [ ]:
# params
data_path = '../data/2013-01-01'
market_name = 'NASDAQ' # 'NASDAQ' 'NYSE'
tickers_fname = market_name + '_tickers_qualify_dr-0.98_min-5_smooth.csv'
steps = 1

In [ ]:
# load stock data
tickers = np.genfromtxt(os.path.join(data_path, '..', tickers_fname),
                              dtype=str, delimiter='\t', skip_header=False)

print('#tickers selected:', len(tickers))
eod_data, mask_data, gt_data, price_data = \
    load_EOD_data(data_path, market_name, tickers, steps)


#tickers selected: 1737
single EOD data shape: (1245, 6)


In [ ]:
!pip install --upgrade -q jax==0.1.57 jaxlib==0.1.37 numpyro
!pip install yfinance

import jax.numpy as np
from jax import random
import numpyro; numpyro.set_host_device_count(4)
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
assert numpyro.__version__.startswith('0.2.4')

from scipy.stats import norm

In [ ]:
def calc_ar(data, predict_len = 1):
    """AR Model for which parameters are calculated and not learned.

    Parameters:
    data (np.array): Array with training data
    predict_len (int): Number of time-steps for which future prediction in needed

    Returns:
    np.array: Array with predictions of specified length 
    """

    # Calculate psi
    # Source: Page 7 of https://www.math.utah.edu/~zhorvath/ar1.pdf
    num = 0
    for i in range(1, len(data)):
        num += data[i] * data[i-1]

    denom = 0
    for i in range(0, len(data) - 1):
        denom += data[i] * data[i]

    psi = num / denom

    # Find the distribution of epsilons
    epsilon_dist = [0]*(len(data)-1)
    for i in range(1, len(data)):
        epsilon_dist[i-1] = data[i] - psi * data[i - 1]
    epsilon_dist = np.array(epsilon_dist)
    (loc, scale) = norm.fit(epsilon_dist)
  
    # For prediction, sample some epsilons from the distribution
    epsilons = numpyro.sample("epsilons", dist.Normal(loc=loc, scale=scale), rng_key=random.PRNGKey(2), sample_shape=(1, predict_len))[0]

    # Make the predictions
    predictions = [0] * predict_len

    y_prev = data[-1]
    for i in range(0, predict_len):
        y_next =  epsilons[i] + y_prev * psi
        predictions[i] = y_next
        y_prev = y_next
    growth = [((predictions[i]-data[-1])/data[-1])] +[0]*(predict_len-1)
    for i in range(1,predict_len):
        growth[i] = (predictions[i]-predictions[i-1])/predictions[i-1]
    return np.array(growth)
def profit(test,truth):
    total = [0]*len(test[0])
    for i in range(0,len(test[0])):
        best_buy = np.argmax(test[:,i])
        total[i] = 100*(truth[best_buy,i]+1)
    return total
def get_dataset(price_data, n ,growth_data, train_size = 500, test_size = 50):
    
    start_point = n*train_size
    split_point = start_point + train_size
    final_point = split_point + test_size
    if final_point > len(price_data[0]):
        return len(price_data[0]) - start_point
    train = price_data[:,start_point:split_point]
    test = price_data[:,split_point:final_point]
    grow = growth_data[:,split_point:final_point]
    return train,grow
def get_results(train,train_len = 50):
    results = [0]*len(train)
    for i in range(0,len(train)):
      
      results[i]=(calc_ar(train[i],predict_len = train_len))
    print("is_it_calc?")
    return np.array(results)
def total_dataset(price_data,gt_data,train_size = 50):
    count = 0
    actual_total = []
    best_total = []
    while type(get_dataset(price_data,count,gt_data))!= type(1):
        print("count",count)
        dataset = get_dataset(price_data,count,gt_data, test_size = train_size)
        predictions = get_results(dataset[0],train_len = train_size)
        actual_total.append(profit(predictions,dataset[1]))
        best_total.append(profit(dataset[1],dataset[1]))
        count+=1
    return actual_total, best_total

In [ ]:
first_round = total_dataset(price_data, gt_data)

count 0
is_it_calc?
count 1
is_it_calc?
count 2
is_it_calc?
count 3
is_it_calc?
count 4
is_it_calc?
count 5


In [ ]:
dataset = get_dataset(price_data,4,gt_data)
predictions = get_results(dataset[0])
#actual_income = profit(predictions,dataset[1])
#best_income = profit(dataset[1],dataset[1])

is_it_calc?


In [ ]:
103669.37569486132-118125.99133141339

-14456.615636552073

In [ ]:
# 1026 companies (NASDAQ) (will be different for NYSE)
# 1245 time steps

# tickers     (1026,)         : array (string)  
#                             - stock symbol
# eod_data    (1026, 1245, 5) : array (float32) 
#                             - features for a given ticker 
#                               [0] 5-day avg, [1] 10-day avg, [2] 20-day avg, 
#                               [3] 30-day avg, [4] actual stock price (scaled)
# mask_data   (1026, 1245)    : array (float32) 
#                             - masks, not sure what these do 
#                               yet. Expected at least one 0 in the position of
#                               the stock ticker's index, but they're all 1...
# gt_data     (1026, 1245)    : array (float32) 
#                              - ground truths, change in day to day price
# price_data  (1026, 1245)    : array (float32) 
#                             - actual stock price (scaled), same as eod_data[4]

In [ ]:
datasets = [tickers, eod_data, mask_data, gt_data, price_data]

i = 2 # Apple for NASDAQ, AAP for NYSE
for d in datasets:
    print(d.shape, d.dtype)
    print(d[i], '\n')

(1737,) <U5
AAP 

(1737, 1245, 5) float32
[[0.357611 0.357296 0.359589 0.367068 0.361763]
 [0.359497 0.357092 0.359617 0.366186 0.361763]
 [0.361773 0.357885 0.359971 0.365516 0.367402]
 ...
 [0.501996 0.481316 0.457942 0.441529 0.486576]
 [0.498613 0.485303 0.462484 0.443908 0.487124]
 [0.499641 0.490832 0.467439 0.446347 0.500599]] 

(1737, 1245) float32
[1. 1. 1. ... 1. 1. 1.] 

(1737, 1245) float32
[ 0.          0.          0.01558752 ... -0.0360843   0.00112625
  0.02766242] 

(1737, 1245) float32
[0.361763 0.361763 0.367402 ... 0.486576 0.487124 0.500599] 



In [ ]:
gt_data[0]

### Relational Features

In [ ]:
relation_name = 'sector_industry' # 'wikidata' 'sector_industry'
emb_fname = 'NASDAQ_rank_lstm_seq-16_unit-64_2.csv.npy'

In [ ]:
# relation data
rname_tail = {'sector_industry': '_industry_relation.npy',
              'wikidata': '_wiki_relation.npy'}

rel_encoding, rel_mask = load_relation_data(
    os.path.join(data_path, '..', 'relation', relation_name,
                  market_name + rname_tail[relation_name])
)
# print('relation encoding shape:', rel_encoding.shape)
print('relation mask shape:', rel_mask.shape)

embedding = np.load(
    os.path.join(data_path, '..', 'pretrain', emb_fname))
print('embedding shape:', embedding.shape)

In [ ]:
rel_datasets = [rel_encoding, rel_mask, embedding]

i = 2 # Apple
for d in rel_datasets:
    print(d.shape, d.dtype)
    print(d[i], '\n')

# Next Steps

- Figure out how to load pretrained weights by reviewing how the TRSR people did it...[link](https://github.com/fulifeng/Temporal_Relational_Stock_Ranking/blob/master/training/relation_rank_lstm.py)
- Rewrite the GNN in `pytorch`
- Initialize company correlation matrix
- Calculate the return baselines (done)
  - Best Possible Return: For each time step, find stock with highest return and invest completely in that stock
  - Standard Return: If we would have just invested in the full index

## Initializing company correlation matrix

### Attempt 1

In [ ]:
from scipy.stats import pearsonr
data = gt_data
n = 30
from datetime import datetime

time_periods = len(gt_data[0])-(n-1)
class analysis_object:
    def __init__(self, data, n):
        self.data = data
        self.n = n
    def product(self,p):
        print(p)
        if n < 2:
            return []
        stock_count = len(self.data)

        covariance_tensor = np.zeros((stock_count,stock_count))
        #count = 1
        for i in range(0,stock_count):
            for j in range(i+1,stock_count):
                #if ((i*stock_count+j)/(stock_count*stock_count))>0.05*count:
                #    clear_output()
                #    print("We are "+str(5*count)+"% done")
                #    count+=1
                x = self.data[i][p:(p+self.n)]
                y = self.data[j][p:(p+self.n)]
                covariance_tensor[i][j] = pearsonr(x,y)[0]
        return covariance_tensor

In [ ]:
data = analysis_object(gt_data,30)

In [ ]:
import concurrent.futures
import pickle
time_periods = len(gt_data[0])-(30-1)
futures = {}
company_correlations = {}
file_name = 'company_correlations.pkl'
with open(file_name, 'wb') as handle:
  with concurrent.futures.ThreadPoolExecutor() as executor:
      for i in range(0,time_periods):
          futures[i] = executor.submit(data.product, i)
      for k in futures:
          company_correlations[k] = futures[k].result()
          if k%5 == 1:
            pickle.dump(company_correlations, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(company_correlations[0])

In [ ]:
company_correlations

In [ ]:
# Code to upload the pickled file to google drive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': folder_id}]})
file.SetContentFile(file_name)
file.Upload() 

### Attempt 2

In [ ]:
from scipy.stats import pearsonr
from tqdm import tqdm

In [ ]:
# corr tensor
corr_size = 30
num_companies, num_timesteps = gt_data.shape
correlation_matrix_shape = (num_timesteps - corr_size, num_companies, num_companies)
corr = np.ones(correlation_matrix_shape)
print(corr.shape)

In [ ]:
for t in tqdm(range(num_timesteps - corr_size)):
    for c1 in tqdm(range(num_companies)):
        for c2 in range(1, num_companies - c1):
            c1_movements = gt_data.T[t:t+corr_size][:,c1]
            c2_movements = gt_data.T[t:t+corr_size][:,c1 + c2]
            c1_c2_corr = pearsonr(c1_movements, c2_movements)[0]
            if np.isnan(c1_c2_corr):
                c1_c2_corr = 1e-10
            corr[t][c1][c1 + c2] = c1_c2_corr        
    corr[t][il] = corr[t][iu]

## Calculating Return Baselines

### Optimal
The best possible return is calculated by finding the stock with the highest 
percent increase at each timestep and investing a fixed, non-compounding amount into that one stock per timestep.

| skip_n_steps | NASDAQ | NYSE |
|---|---|---|
| 0 | 20102.418 | 16884.516 | 
| 30 | 19622.406 | 16482.426 |

In [ ]:
# gt_data.shape  # (1026, 1245)
best_stock_per_t  = np.argmax(gt_data.T, axis=1) # best stock at each timestep
best_return_per_t = np.max(gt_data.T, axis=1) # best return at each timestep

print(best_stock_per_t)
print(best_return_per_t)

In [ ]:
total_return = 0
daily_investment = 100
n = 0 # starting point intended to exclude the first n days we don't have correlations for

In [ ]:
# loop form
for r in best_return_per_t[n:]:
    daily_return = (daily_investment * (1.0 + r)) - daily_investment
    total_return += daily_return
print(total_return)

# vector form (more concise, faster)
total_return = np.sum(np.multiply(daily_investment, best_return_per_t[n:]))
print(total_return)

In [ ]:
def best_return(pct_returns, daily_investment=100, skip_n_steps=0):
    return np.sum(np.multiply(daily_investment, np.max(pct_returns.T, axis=1)[skip_n_steps:]))

print(best_return(gt_data, 100, 0))
print(best_return(gt_data, 100, 30))

16884.516
16482.426


### Average
The average return assumes investing in each stock in the index equally (computed by averaging all the stock returns) and investing a fixed, non-compounding amount at every time step. 

| skip_n_steps | NASDAQ | NYSE |
|---|---|---|
| 0 | 57.427822 | 35.447433 | 
| 30 | 55.723846 | 31.19346 |

In [ ]:
# gt_data.shape  # (1026, 1245)
avg_return_per_t = np.average(gt_data.T, axis=1) # avg return at each timestep
print(avg_return_per_t)

In [ ]:
total_return = 0
daily_investment = 100
n = 0 # starting point intended to exclude the first n days we don't have correlations for

In [ ]:
# loop form
for r in avg_return_per_t[n:]:
    daily_return = (daily_investment * (1.0 + r)) - daily_investment
    total_return += daily_return
print(total_return)

# vector form (more concise, faster)
total_return = np.sum(np.multiply(daily_investment, avg_return_per_t[n:]))
print(total_return)

In [ ]:
def avg_return(pct_returns, daily_investment=100, skip_n_steps=0):
    return np.sum(np.multiply(daily_investment, np.average(pct_returns.T, axis=1)[skip_n_steps:]))

print(avg_return(gt_data, 100, 0))
print(avg_return(gt_data, 100, 30))

35.447433
31.19346


In [ ]:
from time import time

In [ ]:
time()

1592256866.6780775

# Pytorch Examples

Here are some examples of how to make custom models in Pytorch to get you started.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

device = torch.device("cpu")
if torch.cuda.is_available():
    print('CUDA is available!')
    device = torch.device("cuda")
    use_cuda = True
else:
    print('CUDA is not available.')

### Define Models

In [ ]:
class FCNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.dens1 = nn.Linear(784, 256)
        self.relu1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.2)
        self.dens2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.drop2 = nn.Dropout(0.2)
        self.dens3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.drop3 = nn.Dropout(0.2)
        self.dens4 = nn.Linear(64, 20)
        self.relu4 = nn.ReLU()
        self.drop4 = nn.Dropout(0.2)
        self.dens5 = nn.Linear(20, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.dens1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        x = self.dens2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        x = self.dens3(x)
        x = self.relu3(x)
        x = self.drop3(x)
        x = self.dens4(x)
        x = self.relu4(x)
        x = self.drop4(x)
        x = self.dens5(x)
        return F.log_softmax(x, dim=1)

class Conv2DNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.relu2 = nn.ReLU()
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = F.max_pool2d(x, 2, 2)
        x = self.conv2(x)
        x = self.relu2(x)
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Define Training and Testing functions

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        # calculate robust loss
        loss = F.cross_entropy(model(data), target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            batch_size = len(data)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), acc))
    return acc

### Load Data

In [ ]:
data_dir = "/content/data/"
batch_size_train = 10
batch_size_test = 10

train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(data_dir, train=True, download=True,
                         transform=torchvision.transforms.Compose([
                           torchvision.transforms.ToTensor()
                         ])),
    batch_size=batch_size_test, shuffle=False, pin_memory=True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST(data_dir, train=False, download=True,
                         transform=torchvision.transforms.Compose([
                           torchvision.transforms.ToTensor()
                         ])),
    batch_size=batch_size_test, shuffle=False, pin_memory=True)

### Train

In [ ]:
import torch
import numpy as np

In [ ]:
rel_encoding = (torch.ones((100)) * np.arange(100)).view(10,10).unsqueeze_(-1)
rel_shape = rel_encoding.shape

In [ ]:
rel_mask = torch.where(rel_encoding == 0, torch.ones(rel_shape) * -1e9, torch.zeros(rel_shape)).squeeze_()
rel_mask.shape

torch.Size([10, 10])

In [ ]:
n_epochs = 10

fcnet5 = FCNet5().to(device)
conv2dnet = Conv2DNet().to(device)

for model in [fcnet5, conv2dnet]:
    
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

    m_type = model.__class__.__name__
    
    print('training for model', m_type)
    
    for epoch in range(1, n_epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        acc = test(model, device, test_loader)  

    # torch.save(model.state_dict(), save_dir + 'model_' + m_type + '_' + str(datetime.date.today()) + '_' + str(acc) + '.pth')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym

# hyper-parameters
BATCH_SIZE = 128
LR = 0.01
GAMMA = 0.90
EPISILO = 0.9
MEMORY_CAPACITY = 20000
Q_NETWORK_ITERATION = 100

env = gym.make("CartPole-v0")
env = env.unwrapped
NUM_ACTIONS = env.action_space.n
NUM_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample.shape
class Net(nn.Module):
    """docstring for Net"""
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(NUM_STATES, 50)
        self.fc1.weight.data.normal_(0,0.1)
        self.fc2 = nn.Linear(50,30)
        self.fc2.weight.data.normal_(0,0.1)
        self.out = nn.Linear(30,NUM_ACTIONS)
        self.out.weight.data.normal_(0,0.1)

    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        action_prob = self.out(x)
        return action_prob

class DQN():
    """docstring for DQN"""
    def __init__(self):
        super(DQN, self).__init__()
        self.eval_net, self.target_net = Net(), Net()

        self.learn_step_counter = 0
        self.memory_counter = 0
        self.memory = np.zeros((MEMORY_CAPACITY, NUM_STATES * 2 + 2))
        # why the NUM_STATE*2 +2
        # When we store the memory, we put the state, action, reward and next_state in the memory
        # here reward and action is a number, state is a ndarray
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()

    def choose_action(self, state):
        state = torch.unsqueeze(torch.FloatTensor(state), 0) # get a 1D array
        if np.random.randn() <= EPISILO:# greedy policy
            action_value = self.eval_net.forward(state)
            action = torch.max(action_value, 1)[1].data.numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        else: # random policy
            action = np.random.randint(0,NUM_ACTIONS)
            action = action if ENV_A_SHAPE ==0 else action.reshape(ENV_A_SHAPE)
        return action


    def store_transition(self, state, action, reward, next_state):
        transition = np.hstack((state, [action, reward], next_state))
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1


    def learn(self):

        #update the parameters
        if self.learn_step_counter % Q_NETWORK_ITERATION ==0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter+=1

        #sample batch from memory
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        batch_memory = self.memory[sample_index, :]
        batch_state = torch.FloatTensor(batch_memory[:, :NUM_STATES])
        batch_action = torch.LongTensor(batch_memory[:, NUM_STATES:NUM_STATES+1].astype(int))
        batch_reward = torch.FloatTensor(batch_memory[:, NUM_STATES+1:NUM_STATES+2])
        batch_next_state = torch.FloatTensor(batch_memory[:,-NUM_STATES:])

        #q_eval
        q_eval = self.eval_net(batch_state).gather(1, batch_action)
        q_next = self.target_net(batch_next_state).detach()
        q_target = batch_reward + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

def main():
    dqn = DQN()
    episodes = 400
    print("Collecting Experience....")
    for i in range(episodes):
        state = env.reset()
        ep_reward = 0
        while True:
            env.render()
            action = dqn.choose_action(state)
            next_state, reward, done, info = env.step(action)

            dqn.store_transition(state, action, reward, next_state)
            ep_reward += reward

            if dqn.memory_counter >= MEMORY_CAPACITY:
                dqn.learn()
                if done:
                    print("episode: {} , the episode reward is {}".format(i, round(ep_reward, 3)))
            if done:
                break
            state = next_state

if __name__ == '__main__':
    main()